In [1]:
import os
import time
from IPython.display import clear_output
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
# path = os.getcwd()

In [3]:
career_link = pd.read_csv('Career Link Job Details.csv')
career_link.columns

Index(['Job Title', 'Location', 'Salary', 'Experience', 'Company',
       'Company Link', 'Description', 'Benefits', 'Required Skills',
       'Job Summary', 'Contact', 'Keywords'],
      dtype='object')

In [4]:
career_link.head(3)

,Job Title,Location,Salary,Experience,Company,Company Link,Description,Benefits,Required Skills,Job Summary,Contact,Keywords
0,.Net Developer [Huế],"['Thành phố Huế,', 'Thừa Thiên - Huế']",Thương lượng,2 - 5 năm kinh nghiệm,CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,['- Developing Cloud-based application systems...,NaN,['- 2+ years of experience- Knowledge of OOP- ...,"{'Loại công việc': 'Nhân viên toàn thời gian',...","{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...","[['HTML', 'https://www.careerlink.vn//viec-lam..."
1,.Net Developer (Senior/Lead),"['Quận Cầu Giấy,', 'Hà Nội']",Trên 40 triệu,5 - 10 năm kinh nghiệm,CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,['- Làm việc cùng với BA và các bên liên quan ...,"['Cung cấp thiết bị làm việc cho nhân viên', '...",['- Ít nhất 5 năm kinh nghiệm lập trình sử dụn...,"{'Loại công việc': 'Nhân viên toàn thời gian',...","{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...","[['Quận Cầu Giấy', 'https://www.careerlink.vn/..."
2,.Net Developer [Tân Bình],"['Quận Tân Bình,', 'Hồ Chí Minh']",Thương lượng,2 - 5 năm kinh nghiệm,CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,['- Developing Cloud-based application systems...,NaN,['- 2+ years of experience- Knowledge of OOP- ...,"{'Loại công việc': 'Nhân viên toàn thời gian',...","{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...","[['HTML', 'https://www.careerlink.vn//viec-lam..."


In [5]:
career_link.shape

(22800, 12)

In [6]:
career_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Job Title        22800 non-null  object
 1   Location         22800 non-null  object
 2   Salary           22800 non-null  object
 3   Experience       22800 non-null  object
 4   Company          22800 non-null  object
 5   Company Link     22800 non-null  object
 6   Description      22800 non-null  object
 7   Benefits         7513 non-null   object
 8   Required Skills  22800 non-null  object
 9   Job Summary      22800 non-null  object
 10  Contact          22800 non-null  object
 11  Keywords         22800 non-null  object
dtypes: object(12)
memory usage: 2.1+ MB


# Process the Job Summary column

## Tạo thêm cột mới từ nội dung của cột Job Summary

**Biết rằng nội dung của một Job Summary sẽ trông như thế này**

{
  
  'Loại công việc': 'Nhân viên toàn thời gian',

  'Cấp bậc': 'Trưởng nhóm / Giám sát', 

  'Học vấn': 'Cử nhân', 

  'Giới tính': 'Nam / Nữ', 

  'Ngành nghề': ['CNTT - Phần mềm', 'CNTT - Phần cứng / Mạng']
  
}

In [7]:
import pandas as pd
import ast


def extract_keys_and_values(row):
    try:
        job_summary_dict = ast.literal_eval(row['Job Summary'])
        for key, value in job_summary_dict.items():
            row[key] = value
    except (ValueError, SyntaxError):
        pass
    return row

career_link = career_link.apply(extract_keys_and_values, axis=1)

career_link.head()

,Benefits,Company,Company Link,Contact,Cấp bậc,Description,Experience,Giới tính,Học vấn,Job Summary,Job Title,Keywords,Location,Loại công việc,Mã việc làm,Ngành nghề,Required Skills,Salary,Tuổi
0,NaN,CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...",Nhân viên,['- Developing Cloud-based application systems...,2 - 5 năm kinh nghiệm,Nam / Nữ,Cao đẳng,"{'Loại công việc': 'Nhân viên toàn thời gian',...",.Net Developer [Huế],"[['HTML', 'https://www.careerlink.vn//viec-lam...","['Thành phố Huế,', 'Thừa Thiên - Huế']",Nhân viên toàn thời gian,NaN,[CNTT - Phần mềm],['- 2+ years of experience- Knowledge of OOP- ...,Thương lượng,NaN
1,"['Cung cấp thiết bị làm việc cho nhân viên', '...",CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...",Trưởng nhóm / Giám sát,['- Làm việc cùng với BA và các bên liên quan ...,5 - 10 năm kinh nghiệm,Nam / Nữ,Cử nhân,"{'Loại công việc': 'Nhân viên toàn thời gian',...",.Net Developer (Senior/Lead),"[['Quận Cầu Giấy', 'https://www.careerlink.vn/...","['Quận Cầu Giấy,', 'Hà Nội']",Nhân viên toàn thời gian,NaN,"[CNTT - Phần mềm, CNTT - Phần cứng / Mạng]",['- Ít nhất 5 năm kinh nghiệm lập trình sử dụn...,Trên 40 triệu,NaN
2,NaN,CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...",Nhân viên,['- Developing Cloud-based application systems...,2 - 5 năm kinh nghiệm,Nam / Nữ,Cao đẳng,"{'Loại công việc': 'Nhân viên toàn thời gian',...",.Net Developer [Tân Bình],"[['HTML', 'https://www.careerlink.vn//viec-lam...","['Quận Tân Bình,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,NaN,[CNTT - Phần mềm],['- 2+ years of experience- Knowledge of OOP- ...,Thương lượng,NaN
3,NaN,CÔNG TY CỔ PHẦN GIẢI PHÁP KINH DOANH NỀN TẢNG,https://www.careerlink.vn//viec-lam-cua/cong-t...,"{'Contact': ['Tên liên hệ:', 'Phòng Hành Chánh...",Nhân viên,['- Triển khai tích hợp phần mềm - Phân tích v...,2 - 5 năm kinh nghiệm,Nam / Nữ,Cao đẳng,"{'Mã việc làm': '07102020', 'Loại công việc': ...",.NET DEVELOPER,"[['.Net', 'https://www.careerlink.vn//viec-lam...","['Quận 4,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,07102020,"[CNTT - Phần mềm, CNTT - Phần cứng / Mạng]",['- Ít nhất 02 năm kinh nghiệm về phát triển p...,Trên 600 USD,[]
4,NaN,Rieker Vietnam Ltd.,https://www.careerlink.vn//viec-lam-cua/rieker...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],...",Nhân viên,"['- Quản trị cơ sở dữ liệu, phần mềm máy chủ.'...",2 - 5 năm kinh nghiệm,Nam / Nữ,Cử nhân,"{'Loại công việc': 'Nhân viên toàn thời gian',...",01 Chuyên Viên Quản Trị cơ sở dữ liệu/Database...,"[['Phần cứng', 'https://www.careerlink.vn//vie...","['Thị xã Điện Bàn,', 'Quảng Nam']",Nhân viên toàn thời gian,NaN,"[CNTT - Phần mềm, CNTT - Phần cứng / Mạng]","['- Tốt nghiệp Đại học công nghệ thông tin, qu...",Thương lượng,NaN


In [8]:
career_link.columns

Index(['Benefits', 'Company', 'Company Link', 'Contact', 'Cấp bậc',
       'Description', 'Experience', 'Giới tính', 'Học vấn', 'Job Summary',
       'Job Title', 'Keywords', 'Location', 'Loại công việc', 'Mã việc làm',
       'Ngành nghề', 'Required Skills', 'Salary', 'Tuổi'],
      dtype='object')

## Đổi tên các cột thành tiếng Việt thành tiếng Anh

In [9]:
column_name_mapping = {
    'Cấp bậc': 'Level',
    'Giới tính': 'Gender',
    'Học vấn': 'Education',
    'Loại công việc': 'Job Type',
    'Ngành nghề': 'Industry',
    'Tuổi': 'Age',
}

career_link.rename(columns=column_name_mapping, inplace=True)

career_link.columns

Index(['Benefits', 'Company', 'Company Link', 'Contact', 'Level',
       'Description', 'Experience', 'Gender', 'Education', 'Job Summary',
       'Job Title', 'Keywords', 'Location', 'Job Type', 'Mã việc làm',
       'Industry', 'Required Skills', 'Salary', 'Age'],
      dtype='object')

## Loại bỏ cột 'Mã việc làm' và sắp xếp lại thứ tự các cột cho dễ đọc

In [10]:
career_link.drop(columns=['Mã việc làm'], inplace=True)

desired_column_order = [
    'Job Title',
    'Company',
    'Company Link',
    'Location',
    'Job Type',
    'Industry',
    'Level',
    'Experience',
    'Salary',
    'Benefits',
    'Education',
    'Age',
    'Gender',
    'Keywords',
    'Job Summary',
    'Description',
    'Required Skills',
    'Contact',
]

career_link = career_link[desired_column_order]
career_link.head()

,Job Title,Company,Company Link,Location,Job Type,Industry,Level,Experience,Salary,Benefits,Education,Age,Gender,Keywords,Job Summary,Description,Required Skills,Contact
0,.Net Developer [Huế],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Thành phố Huế,', 'Thừa Thiên - Huế']",Nhân viên toàn thời gian,[CNTT - Phần mềm],Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,NaN,Cao đẳng,NaN,Nam / Nữ,"[['HTML', 'https://www.careerlink.vn//viec-lam...","{'Loại công việc': 'Nhân viên toàn thời gian',...",['- Developing Cloud-based application systems...,['- 2+ years of experience- Knowledge of OOP- ...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],..."
1,.Net Developer (Senior/Lead),CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Cầu Giấy,', 'Hà Nội']",Nhân viên toàn thời gian,"[CNTT - Phần mềm, CNTT - Phần cứng / Mạng]",Trưởng nhóm / Giám sát,5 - 10 năm kinh nghiệm,Trên 40 triệu,"['Cung cấp thiết bị làm việc cho nhân viên', '...",Cử nhân,NaN,Nam / Nữ,"[['Quận Cầu Giấy', 'https://www.careerlink.vn/...","{'Loại công việc': 'Nhân viên toàn thời gian',...",['- Làm việc cùng với BA và các bên liên quan ...,['- Ít nhất 5 năm kinh nghiệm lập trình sử dụn...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],..."
2,.Net Developer [Tân Bình],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Tân Bình,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,[CNTT - Phần mềm],Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,NaN,Cao đẳng,NaN,Nam / Nữ,"[['HTML', 'https://www.careerlink.vn//viec-lam...","{'Loại công việc': 'Nhân viên toàn thời gian',...",['- Developing Cloud-based application systems...,['- 2+ years of experience- Knowledge of OOP- ...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],..."
3,.NET DEVELOPER,CÔNG TY CỔ PHẦN GIẢI PHÁP KINH DOANH NỀN TẢNG,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận 4,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"[CNTT - Phần mềm, CNTT - Phần cứng / Mạng]",Nhân viên,2 - 5 năm kinh nghiệm,Trên 600 USD,NaN,Cao đẳng,[],Nam / Nữ,"[['.Net', 'https://www.careerlink.vn//viec-lam...","{'Mã việc làm': '07102020', 'Loại công việc': ...",['- Triển khai tích hợp phần mềm - Phân tích v...,['- Ít nhất 02 năm kinh nghiệm về phát triển p...,"{'Contact': ['Tên liên hệ:', 'Phòng Hành Chánh..."
4,01 Chuyên Viên Quản Trị cơ sở dữ liệu/Database...,Rieker Vietnam Ltd.,https://www.careerlink.vn//viec-lam-cua/rieker...,"['Thị xã Điện Bàn,', 'Quảng Nam']",Nhân viên toàn thời gian,"[CNTT - Phần mềm, CNTT - Phần cứng / Mạng]",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,NaN,Cử nhân,NaN,Nam / Nữ,"[['Phần cứng', 'https://www.careerlink.vn//vie...","{'Loại công việc': 'Nhân viên toàn thời gian',...","['- Quản trị cơ sở dữ liệu, phần mềm máy chủ.'...","['- Tốt nghiệp Đại học công nghệ thông tin, qu...","{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],..."


# Loại bỏ cột duplicate

In [11]:
# Convert all lists in the DataFrame to tuples
career_link = career_link.applymap(lambda x: tuple(x) if isinstance(x, list) else x)

career_link.drop_duplicates(inplace=True)

# Check the shape of the DataFrame after dropping duplicates
print(career_link.shape)


(22800, 18)


## Kiểm tra giá trị một số cột đặc biệt và loại bỏ nếu cần

In [12]:
career_link['Age'].explode().unique()

array([nan], dtype=object)

In [13]:
career_link.drop(columns=['Age'], inplace=True)

In [14]:
career_link['Industry'].explode().unique()

array(['CNTT - Phần mềm', 'CNTT - Phần cứng / Mạng',
       'Giáo dục / Đào tạo / Thư viện',
       'Biên phiên dịch / Thông dịch viên',
       'Biên phiên dịch (tiếng Nhật)', 'Xuất nhập khẩu / Ngoại thương',
       'Vận chuyển / Giao thông / Kho bãi', 'An Ninh / Bảo Vệ',
       'Thư ký / Hành chánh', 'Kỹ thuật ứng dụng / Cơ khí',
       'Điện / Điện tử', 'Bảo trì / Sửa chữa',
       'Nghệ thuật / Thiết kế / Giải trí', 'Sản xuất / Vận hành sản xuất',
       'Bán hàng / Kinh doanh', 'Nhân sự', 'Đồ Gỗ',
       'Nội thất / Ngoại thất', 'Hóa chất / Sinh hóa / Thực phẩm',
       'Bán lẻ / Bán sỉ', 'Dệt may / Da giày',
       'Quản lý chất lượng (QA / QC)', 'Vật tư / Thu mua', 'Bất động sản',
       'Tiếp thị', 'Dịch vụ khách hàng', 'Tư vấn',
       'Mới tốt nghiệp / Thực tập', 'Lao động phổ thông', 'Kiến trúc',
       'Kế toán / Kiểm toán', 'Quản lý điều hành',
       'Điện lạnh / Nhiệt lạnh', 'Quảng cáo / Khuyến mãi / Đối ngoại',
       'Báo chí / Biên tập viên / Xuất bản', 'Chăm sóc sức k

In [15]:
career_link['Gender'].unique()

array(['Nam / Nữ', 'Nam', 'Nữ'], dtype=object)

In [16]:
career_link['Education'].unique()

array(['Cao đẳng', 'Cử nhân', 'Thạc sĩ', 'Trung cấp', 'Kỹ sư',
       'Trung học phổ thông', 'Chứng chỉ', 'Trung học cơ sở',
       'Thạc sĩ Y học', 'Khác', 'Tiến sĩ', 'Thạc sĩ Nghệ thuật',
       'Thạc sĩ QTKD'], dtype=object)

In [17]:
career_link['Experience'].unique()

array(['2 - 5 năm kinh nghiệm', '5 - 10 năm kinh nghiệm',
       '1 - 2 năm kinh nghiệm', '0 - 1 năm kinh nghiệm',
       'Hơn 10 năm kinh nghiệm'], dtype=object)

In [18]:
career_link['Salary'].unique()

array(['Thương lượng', 'Trên 40 triệu', 'Trên 600 USD',
       '8 triệu - 10 triệu', '15 triệu - 22 triệu', 'Cạnh tranh',
       '7 triệu - 15 triệu', '7 triệu - 10 triệu', '20 triệu - 80 triệu',
       '8.5 triệu - 10 triệu', '7.5 triệu - 10 triệu',
       '10 triệu - 15 triệu', '9 triệu - 20 triệu', '2,000 USD',
       'Trên 1,200 USD', '1,000 USD', '8 triệu - 15 triệu',
       '8 triệu - 12 triệu', '7 triệu - 8 triệu', '7 triệu - 9 triệu',
       '10 triệu - 13 triệu', '9 triệu - 13 triệu',
       '10 triệu - 13.5 triệu', '6 triệu - 15 triệu',
       '20 triệu - 35 triệu', '14 triệu - 20 triệu',
       '12 triệu - 15 triệu', '18 triệu - 25 triệu',
       '6.3 triệu - 10 triệu', '7 triệu - 12 triệu', 'Trên 3,500 USD',
       '7 triệu - 21 triệu', '7 triệu - 13 triệu', 'Trên 7 triệu',
       '10 triệu - 20 triệu', '1,000 USD - 1,500 USD',
       '20 triệu - 30 triệu', '12 triệu - 20 triệu', '4 triệu - 8 triệu',
       '10 triệu - 22 triệu', '20 triệu - 24 triệu',
       '12 triệu - 25

In [19]:
career_link['Job Type'].unique()

array(['Nhân viên toàn thời gian', 'Nhân viên toàn thời gian tạm thời',
       'Nhân viên bán thời gian', 'Nhân viên hợp đồng', 'Khác',
       'Nhân viên bán thời gian tạm thời'], dtype=object)

In [20]:
career_link.head(2)

,Job Title,Company,Company Link,Location,Job Type,Industry,Level,Experience,Salary,Benefits,Education,Gender,Keywords,Job Summary,Description,Required Skills,Contact
0,.Net Developer [Huế],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Thành phố Huế,', 'Thừa Thiên - Huế']",Nhân viên toàn thời gian,"(CNTT - Phần mềm,)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,NaN,Cao đẳng,Nam / Nữ,"[['HTML', 'https://www.careerlink.vn//viec-lam...","{'Loại công việc': 'Nhân viên toàn thời gian',...",['- Developing Cloud-based application systems...,['- 2+ years of experience- Knowledge of OOP- ...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],..."
1,.Net Developer (Senior/Lead),CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Cầu Giấy,', 'Hà Nội']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Trưởng nhóm / Giám sát,5 - 10 năm kinh nghiệm,Trên 40 triệu,"['Cung cấp thiết bị làm việc cho nhân viên', '...",Cử nhân,Nam / Nữ,"[['Quận Cầu Giấy', 'https://www.careerlink.vn/...","{'Loại công việc': 'Nhân viên toàn thời gian',...",['- Làm việc cùng với BA và các bên liên quan ...,['- Ít nhất 5 năm kinh nghiệm lập trình sử dụn...,"{'Contact': ['Tên liên hệ:', 'Phòng Nhân Sự'],..."


# Làm việc với cột lương

In [21]:
def process_salary_range(row):
    def convert_to_num(salary_str):
        cleaned_str = re.sub(r'[^\d.,]', '', salary_str)
        if 'triệu' in salary_str:
            amount = float(cleaned_str.split()[0].replace(',', '.'))
            return amount * 1_000_000
        elif 'USD' in salary_str:
            amount = float(cleaned_str.split()[0].replace(',', ''))
            return amount
        else:
            return None

    if isinstance(row, str):
        if ' - ' in row:
            parts = row.split('-')
            lower_limit = convert_to_num(parts[0].strip())
            upper_limit = convert_to_num(parts[1].strip())
            return (lower_limit, upper_limit)
        elif 'Trên' in row:
            row = re.sub(r'Trên ', '', row)
            lower_limit = convert_to_num(row.strip())
            return (lower_limit, None)
        elif re.match(r'^\d{1,3}(,\d{3})*(\.\d+)? (triệu|USD)$', row):
            value = convert_to_num(row)
            return (value, value)
    return (None, None)


career_link[['Salary Start', 'Salary End']] = career_link['Salary'].apply(process_salary_range).apply(pd.Series)
career_link['Salary Type'] = career_link['Salary'].apply(lambda x: 'Defined' if any(char.isdigit() for char in str(x)) else 'Undefined')

def categorize_undefined_salary(row):
    if row['Salary Type'] == 'Undefined':
        return row['Salary']
    else:
        return None

career_link['Undefined Salary Type'] = career_link.apply(categorize_undefined_salary, axis=1)
career_link['Currency Unit'] = career_link.apply(lambda row: 'USD' if row['Salary Type'] == 'Defined' and 'USD' in str(row['Salary']) else ('VND' if row['Salary Type'] == 'Defined' else None), axis=1)
career_link.tail(3)

,Job Title,Company,Company Link,Location,Job Type,Industry,Level,Experience,Salary,Benefits,...,Keywords,Job Summary,Description,Required Skills,Contact,Salary Start,Salary End,Salary Type,Undefined Salary Type,Currency Unit
22797,[Xuyên Mộc] - Chuyên Viên Tư Vấn Bảo Hiểm Kênh...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Bán hàng /...",Nhân viên,0 - 1 năm kinh nghiệm,Cạnh tranh,NaN,...,"[['Ngân hàng', 'https://www.careerlink.vn//vie...","{'Mã việc làm': 'ĐTT003', 'Loại công việc': 'N...",['- Là đại diện của Công ty BHNT Manulife Viet...,"['- Nam/Nữ, tuổi từ 20-35\n \n - Tốt n...","{'Contact': ['Tên liên hệ:', 'Ms.Thương'], 'Lo...",NaN,NaN,Undefined,Cạnh tranh,None
22798,[Xuyên Mộc] Chuyên Viên Tư Vấn Bảo Hiểm Kênh L...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Bán hàng /...",Nhân viên,0 - 1 năm kinh nghiệm,10 triệu - 20 triệu,NaN,...,"[['Nhân Viên Kinh Doanh', 'https://www.careerl...","{'Loại công việc': 'Nhân viên toàn thời gian',...",['- Là đại diện của Công ty BHNT Manulife Viet...,"['- Nam/Nữ, tuổi từ 20-35\n \n - Tốt n...","{'Contact': ['Tên liên hệ:', 'Ms Hiền'], 'Loca...",10000000.0,20000000.0,Defined,None,VND
22799,[Yên Bái] Công Nhân Chăn Nuôi Heo,Công Ty TNHH Japfa Comfeed Việt Nam,https://www.careerlink.vn//viec-lam-cua/cong-t...,['Yên Bái'],Nhân viên toàn thời gian,"(Lao động phổ thông,)",Nhân viên,0 - 1 năm kinh nghiệm,6.9 triệu - 7.5 triệu,"['Bố trí ăn, ở miễn phí', 'có phòng riêng cho ...",...,"[['Lao động', 'https://www.careerlink.vn//viec...","{'Loại công việc': 'Nhân viên toàn thời gian',...","['- Cho lợn ăn theo quy trình, theo dõi tình h...","['- Nam/nữ, tuổi 18 trở lên, sức khỏe tốt.', '...","{'Contact': ['Tên liên hệ:', 'Mr Nam'], 'Locat...",6900000.0,7500000.0,Defined,None,VND


In [22]:
desired_column_order = [
    'Job Title', 'Company', 'Company Link', 'Location', 'Job Type', 'Industry',
    'Level', 'Experience', 'Salary', 'Salary Type',
    'Undefined Salary Type', 'Salary Start', 'Salary End', 'Currency Unit', 'Benefits', 'Education', 'Gender',
    'Keywords', 'Job Summary', 'Description', 'Required Skills', 'Contact'
]

career_link = career_link[desired_column_order]

In [23]:
career_link.columns

Index(['Job Title', 'Company', 'Company Link', 'Location', 'Job Type',
       'Industry', 'Level', 'Experience', 'Salary', 'Salary Type',
       'Undefined Salary Type', 'Salary Start', 'Salary End', 'Currency Unit',
       'Benefits', 'Education', 'Gender', 'Keywords', 'Job Summary',
       'Description', 'Required Skills', 'Contact'],
      dtype='object')

# Làm việc với cột "Industry"

In [24]:
# Đảm bảo rằng mỗi phần tử trong cột 'Industry' là một chuỗi hợp lệ để đánh giá
career_link['Industry'] = career_link['Industry'].apply(lambda x: eval(x) if isinstance(x, str) else x)


In [25]:
merged_industries = [item for sublist in career_link['Industry'] for item in sublist]
all_industries = list(set(merged_industries))
len(all_industries)

57

In [26]:
all_industries[:20]

['CNTT - Phần mềm',
 'Đồ Gỗ',
 'Giáo dục / Đào tạo / Thư viện',
 'Điện / Điện tử',
 'Vận chuyển / Giao thông / Kho bãi',
 'Thuỷ Hải Sản',
 'Mới tốt nghiệp / Thực tập',
 'Nội thất / Ngoại thất',
 'Môi trường / Xử lý chất thải',
 'Lao động phổ thông',
 'Hóa chất / Sinh hóa / Thực phẩm',
 'Kế toán / Kiểm toán',
 'Dược / Sinh học',
 'Bất động sản',
 'Điện lạnh / Nhiệt lạnh',
 'Tài chính / Đầu tư',
 'Biên phiên dịch (tiếng Nhật)',
 'Nghệ thuật / Thiết kế / Giải trí',
 'CNTT - Phần cứng / Mạng',
 'Dệt may / Da giày']

In [27]:
for industry in all_industries:
    career_link[f'Industry_{industry}'] = career_link['Industry'].apply(lambda x: 1 if industry in x else 0)

In [28]:
career_link.shape

(22800, 79)

In [29]:
career_link.iloc[10]['Industry']

('Nghệ thuật / Thiết kế / Giải trí', 'Sản xuất / Vận hành sản xuất')

In [30]:
career_link.iloc[:]

,Job Title,Company,Company Link,Location,Job Type,Industry,Level,Experience,Salary,Salary Type,...,Industry_Quản lý điều hành,Industry_Báo chí / Biên tập viên / Xuất bản,Industry_Ngân hàng / Chứng khoán,Industry_Xuất nhập khẩu / Ngoại thương,Industry_Bảo trì / Sửa chữa,Industry_Khách sạn,Industry_Khác,Industry_Thư ký / Hành chánh,Industry_Vật tư / Thu mua,Industry_Viễn Thông
0,.Net Developer [Huế],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Thành phố Huế,', 'Thừa Thiên - Huế']",Nhân viên toàn thời gian,"(CNTT - Phần mềm,)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,Undefined,...,0,0,0,0,0,0,0,0,0,0
1,.Net Developer (Senior/Lead),CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Cầu Giấy,', 'Hà Nội']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Trưởng nhóm / Giám sát,5 - 10 năm kinh nghiệm,Trên 40 triệu,Defined,...,0,0,0,0,0,0,0,0,0,0
2,.Net Developer [Tân Bình],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Tân Bình,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"(CNTT - Phần mềm,)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,Undefined,...,0,0,0,0,0,0,0,0,0,0
3,.NET DEVELOPER,CÔNG TY CỔ PHẦN GIẢI PHÁP KINH DOANH NỀN TẢNG,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận 4,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Nhân viên,2 - 5 năm kinh nghiệm,Trên 600 USD,Defined,...,0,0,0,0,0,0,0,0,0,0
4,01 Chuyên Viên Quản Trị cơ sở dữ liệu/Database...,Rieker Vietnam Ltd.,https://www.careerlink.vn//viec-lam-cua/rieker...,"['Thị xã Điện Bàn,', 'Quảng Nam']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,Undefined,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,Xuất nhập khẩu tiếng Nhật,G.A. CONSULTANTS VIETNAM,https://www.careerlink.vn//viec-lam-cua/g.a.-c...,"['Quận 7,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"(Sản xuất / Vận hành sản xuất, Bán hàng / Kinh...",Nhân viên,2 - 5 năm kinh nghiệm,15 triệu - 25 triệu,Defined,...,0,0,0,0,0,0,0,0,0,0
22796,[XUYÊN MỘC] CHUYÊN VIÊN TƯ VẤN BẢO HIÊM- DỰ ÁN...,Sun Life Việt Nam (SLV),https://www.careerlink.vn//viec-lam-cua/sun-li...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Tài chính ...",Nhân viên,1 - 2 năm kinh nghiệm,Trên 10 triệu,Defined,...,0,0,1,0,0,0,0,0,0,0
22797,[Xuyên Mộc] - Chuyên Viên Tư Vấn Bảo Hiểm Kênh...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Bán hàng /...",Nhân viên,0 - 1 năm kinh nghiệm,Cạnh tranh,Undefined,...,0,0,1,0,0,0,0,0,0,0
22798,[Xuyên Mộc] Chuyên Viên Tư Vấn Bảo Hiểm Kênh L...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Bán hàng /...",Nhân viên,0 - 1 năm kinh nghiệm,10 triệu - 20 triệu,Defined,...,0,0,1,0,0,0,0,0,0,0


# Làm việc với cột Job Type

In [31]:
career_link['Job Type'].unique()

array(['Nhân viên toàn thời gian', 'Nhân viên toàn thời gian tạm thời',
       'Nhân viên bán thời gian', 'Nhân viên hợp đồng', 'Khác',
       'Nhân viên bán thời gian tạm thời'], dtype=object)

In [32]:
unique_job_types = career_link['Job Type'].unique()

for job_type in unique_job_types:
    career_link[f'Job Type_{job_type}'] = (career_link['Job Type'] == job_type).astype(int)
career_link.iloc[:]

,Job Title,Company,Company Link,Location,Job Type,Industry,Level,Experience,Salary,Salary Type,...,Industry_Khác,Industry_Thư ký / Hành chánh,Industry_Vật tư / Thu mua,Industry_Viễn Thông,Job Type_Nhân viên toàn thời gian,Job Type_Nhân viên toàn thời gian tạm thời,Job Type_Nhân viên bán thời gian,Job Type_Nhân viên hợp đồng,Job Type_Khác,Job Type_Nhân viên bán thời gian tạm thời
0,.Net Developer [Huế],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Thành phố Huế,', 'Thừa Thiên - Huế']",Nhân viên toàn thời gian,"(CNTT - Phần mềm,)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,Undefined,...,0,0,0,0,1,0,0,0,0,0
1,.Net Developer (Senior/Lead),CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Cầu Giấy,', 'Hà Nội']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Trưởng nhóm / Giám sát,5 - 10 năm kinh nghiệm,Trên 40 triệu,Defined,...,0,0,0,0,1,0,0,0,0,0
2,.Net Developer [Tân Bình],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận Tân Bình,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"(CNTT - Phần mềm,)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,Undefined,...,0,0,0,0,1,0,0,0,0,0
3,.NET DEVELOPER,CÔNG TY CỔ PHẦN GIẢI PHÁP KINH DOANH NỀN TẢNG,https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Quận 4,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Nhân viên,2 - 5 năm kinh nghiệm,Trên 600 USD,Defined,...,0,0,0,0,1,0,0,0,0,0
4,01 Chuyên Viên Quản Trị cơ sở dữ liệu/Database...,Rieker Vietnam Ltd.,https://www.careerlink.vn//viec-lam-cua/rieker...,"['Thị xã Điện Bàn,', 'Quảng Nam']",Nhân viên toàn thời gian,"(CNTT - Phần mềm, CNTT - Phần cứng / Mạng)",Nhân viên,2 - 5 năm kinh nghiệm,Thương lượng,Undefined,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,Xuất nhập khẩu tiếng Nhật,G.A. CONSULTANTS VIETNAM,https://www.careerlink.vn//viec-lam-cua/g.a.-c...,"['Quận 7,', 'Hồ Chí Minh']",Nhân viên toàn thời gian,"(Sản xuất / Vận hành sản xuất, Bán hàng / Kinh...",Nhân viên,2 - 5 năm kinh nghiệm,15 triệu - 25 triệu,Defined,...,0,0,0,0,1,0,0,0,0,0
22796,[XUYÊN MỘC] CHUYÊN VIÊN TƯ VẤN BẢO HIÊM- DỰ ÁN...,Sun Life Việt Nam (SLV),https://www.careerlink.vn//viec-lam-cua/sun-li...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Tài chính ...",Nhân viên,1 - 2 năm kinh nghiệm,Trên 10 triệu,Defined,...,0,0,0,0,1,0,0,0,0,0
22797,[Xuyên Mộc] - Chuyên Viên Tư Vấn Bảo Hiểm Kênh...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Bán hàng /...",Nhân viên,0 - 1 năm kinh nghiệm,Cạnh tranh,Undefined,...,0,0,0,0,1,0,0,0,0,0
22798,[Xuyên Mộc] Chuyên Viên Tư Vấn Bảo Hiểm Kênh L...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"['Huyện Xuyên Mộc,', 'Bà Rịa - Vũng Tàu']",Nhân viên toàn thời gian,"(Ngân hàng / Chứng khoán, Bảo hiểm, Bán hàng /...",Nhân viên,0 - 1 năm kinh nghiệm,10 triệu - 20 triệu,Defined,...,0,0,0,0,1,0,0,0,0,0


In [33]:
career_link.shape

(22800, 85)

In [34]:
career_link.iloc[1,8:15]

Salary                                                       Trên 40 triệu
Salary Type                                                        Defined
Undefined Salary Type                                                 None
Salary Start                                                    40000000.0
Salary End                                                             NaN
Currency Unit                                                          VND
Benefits                 ['Cung cấp thiết bị làm việc cho nhân viên', '...
Name: 1, dtype: object

# Làm việc với cột Experience

In [35]:
def process_experience_range(row):
    if isinstance(row, str):
        # Define a regular expression pattern to match the format 'X - Y năm kinh nghiệm'
        pattern = r'(\d+)\s*-\s*(\d+)\s'
        match = re.match(pattern, row)
        
        if match:
            lower_limit = int(match.group(1))
            upper_limit = int(match.group(2))
            return (lower_limit, upper_limit)
        
        # Check if the format is 'Hơn X năm kinh nghiệm'
        else:
            pattern = r'(\d+)'
            match = re.search(pattern, row)

            if match:
                lower_limit = int(match.group(1))
                upper_limit = None
                return (lower_limit, upper_limit)
            
    return (None, None)

career_link['Experience Type (year)'] = career_link['Experience'].apply(lambda x: 'Defined' if any(char.isdigit() for char in str(x)) else 'Undefined')
career_link[['Experience Start (year)', 'Experience End']] = career_link['Experience'].apply(process_experience_range).apply(pd.Series)

def categorize_undefined_salary(row):
    if row['Experience Type (year)'] == 'Undefined':
        return row['Experience']
    else:
        return None

career_link['Experience Undefined Type'] = career_link.apply(categorize_undefined_salary, axis=1)

In [36]:
experience_columns = career_link.loc[:, career_link.columns.str.contains('Experience')]
experience_columns.head()

,Experience,Experience Type (year),Experience Start (year),Experience End,Experience Undefined Type
0,2 - 5 năm kinh nghiệm,Defined,2.0,5.0,None
1,5 - 10 năm kinh nghiệm,Defined,5.0,10.0,None
2,2 - 5 năm kinh nghiệm,Defined,2.0,5.0,None
3,2 - 5 năm kinh nghiệm,Defined,2.0,5.0,None
4,2 - 5 năm kinh nghiệm,Defined,2.0,5.0,None


# Thay đổi thứ tự cột :

In [37]:
# Define the desired column order
desired_columns = [
    'Job Title', 'Company', 'Company Link', 'Location',
    'Level', 'Salary',
    'Salary Type', 'Undefined Salary Type', 'Salary Start', 'Salary End', 'Currency Unit', 'Benefits',
    'Education', 'Gender', 'Keywords', 'Job Summary', 'Description', 'Required Skills', 'Contact'
]

# Group 'Job Type' columns together
job_type_columns = [col for col in career_link.columns if col.startswith('Job Type')]
desired_columns.extend(job_type_columns)

# Group 'Experience' columns together
experience_columns = [col for col in career_link.columns if col.startswith('Experience')]
desired_columns.extend(experience_columns)

# Group 'Industry' columns together
industry_columns = [col for col in career_link.columns if col.startswith('Industry')]
desired_columns.extend(industry_columns)

# Print the DataFrame with the desired column order
career_link = career_link[desired_columns]
career_link.columns

Index(['Job Title', 'Company', 'Company Link', 'Location', 'Level', 'Salary',
       'Salary Type', 'Undefined Salary Type', 'Salary Start', 'Salary End',
       'Currency Unit', 'Benefits', 'Education', 'Gender', 'Keywords',
       'Job Summary', 'Description', 'Required Skills', 'Contact', 'Job Type',
       'Job Type_Nhân viên toàn thời gian',
       'Job Type_Nhân viên toàn thời gian tạm thời',
       'Job Type_Nhân viên bán thời gian', 'Job Type_Nhân viên hợp đồng',
       'Job Type_Khác', 'Job Type_Nhân viên bán thời gian tạm thời',
       'Experience', 'Experience Type (year)', 'Experience Start (year)',
       'Experience End', 'Experience Undefined Type', 'Industry',
       'Industry_CNTT - Phần mềm', 'Industry_Đồ Gỗ',
       'Industry_Giáo dục / Đào tạo / Thư viện', 'Industry_Điện / Điện tử',
       'Industry_Vận chuyển / Giao thông / Kho bãi', 'Industry_Thuỷ Hải Sản',
       'Industry_Mới tốt nghiệp / Thực tập', 'Industry_Nội thất / Ngoại thất',
       'Industry_Môi trường /

In [38]:
career_link.shape

(22800, 89)

# Làm việc với cột Gender

In [39]:
career_link['Male'] = career_link['Gender'].apply(lambda x: 1 if 'Nam' in x else 0)
career_link['Female'] = career_link['Gender'].apply(lambda x: 1 if 'Nữ' in x else 0)

In [40]:
career_link.iloc[1][['Gender', 'Female', 'Male']]

Gender    Nam / Nữ
Female           1
Male             1
Name: 1, dtype: object

In [41]:
career_link.shape

(22800, 91)

# Làm việc với cột Level

In [42]:
career_link['Level'].unique()

array(['Nhân viên', 'Trưởng nhóm / Giám sát', 'Kỹ thuật viên / Kỹ sư',
       'Giám đốc', 'Quản lý / Trưởng phòng', 'Mới đi làm',
       'Sinh viên / Thực tập sinh', 'Quản lý cấp cao',
       'Điều hành cấp cao'], dtype=object)

In [43]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(career_link['Level'].values.reshape(-1, 1))

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Level']))
career_link = pd.concat([career_link, one_hot_df], axis=1)

Thay đổi thứ tự cột 

In [44]:
desired_columns = [
    'Job Title', 'Company', 'Company Link', 'Location',
    'Gender', 'Female', 'Male',
    'Salary',
    'Salary Type', 'Undefined Salary Type', 'Salary Start', 'Salary End', 'Currency Unit', 'Benefits',
    'Education', 'Keywords', 'Job Summary', 'Description', 'Required Skills', 'Contact'
]

# Group 'Level' columns together
job_level_columns = [col for col in career_link.columns if col.startswith('Level')]
desired_columns.extend(job_level_columns)

# Group 'Job Type' columns together
job_type_columns = [col for col in career_link.columns if col.startswith('Job Type')]
desired_columns.extend(job_type_columns)

# Group 'Experience' columns together
experience_columns = [col for col in career_link.columns if col.startswith('Experience')]
desired_columns.extend(experience_columns)

# Group 'Industry' columns together
industry_columns = [col for col in career_link.columns if col.startswith('Industry')]
desired_columns.extend(industry_columns)

# Print the DataFrame with the desired column order
career_link = career_link[desired_columns]

In [45]:
for col in career_link.columns:
    print(col)

Job Title
Company
Company Link
Location
Gender
Female
Male
Salary
Salary Type
Undefined Salary Type
Salary Start
Salary End
Currency Unit
Benefits
Education
Keywords
Job Summary
Description
Required Skills
Contact
Level
Level_Giám đốc
Level_Kỹ thuật viên / Kỹ sư
Level_Mới đi làm
Level_Nhân viên
Level_Quản lý / Trưởng phòng
Level_Quản lý cấp cao
Level_Sinh viên / Thực tập sinh
Level_Trưởng nhóm / Giám sát
Level_Điều hành cấp cao
Job Type
Job Type_Nhân viên toàn thời gian
Job Type_Nhân viên toàn thời gian tạm thời
Job Type_Nhân viên bán thời gian
Job Type_Nhân viên hợp đồng
Job Type_Khác
Job Type_Nhân viên bán thời gian tạm thời
Experience
Experience Type (year)
Experience Start (year)
Experience End
Experience Undefined Type
Industry
Industry_CNTT - Phần mềm
Industry_Đồ Gỗ
Industry_Giáo dục / Đào tạo / Thư viện
Industry_Điện / Điện tử
Industry_Vận chuyển / Giao thông / Kho bãi
Industry_Thuỷ Hải Sản
Industry_Mới tốt nghiệp / Thực tập
Industry_Nội thất / Ngoại thất
Industry_Môi trường / X

In [46]:
career_link.shape

(22800, 100)

# Làm việc với cột Education

In [47]:
career_link['Education'].unique()

array(['Cao đẳng', 'Cử nhân', 'Thạc sĩ', 'Trung cấp', 'Kỹ sư',
       'Trung học phổ thông', 'Chứng chỉ', 'Trung học cơ sở',
       'Thạc sĩ Y học', 'Khác', 'Tiến sĩ', 'Thạc sĩ Nghệ thuật',
       'Thạc sĩ QTKD'], dtype=object)

In [48]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(career_link['Education'].values.reshape(-1, 1))

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Education']))

career_link = pd.concat([career_link, one_hot_df], axis=1)

In [49]:
career_link.shape

(22800, 113)

# Làm việc với cột Location

In [50]:
career_link['Location'] = career_link['Location'].apply(eval)

In [51]:
career_link['City'] = career_link['Location'].apply(lambda x: x[-1] if x else None)
career_link[['Location','City']].head(10)

,Location,City
0,"[Thành phố Huế,, Thừa Thiên - Huế]",Thừa Thiên - Huế
1,"[Quận Cầu Giấy,, Hà Nội]",Hà Nội
2,"[Quận Tân Bình,, Hồ Chí Minh]",Hồ Chí Minh
3,"[Quận 4,, Hồ Chí Minh]",Hồ Chí Minh
4,"[Thị xã Điện Bàn,, Quảng Nam]",Quảng Nam
5,"[Thành phố Biên Hòa,, Đồng Nai]",Đồng Nai
6,"[Thành phố Từ Sơn,, Bắc Ninh]",Bắc Ninh
7,"[Thị xã Phú Mỹ,, Bà Rịa - Vũng Tàu]",Bà Rịa - Vũng Tàu
8,"[Thị xã Phú Mỹ,, Bà Rịa - Vũng Tàu]",Bà Rịa - Vũng Tàu
9,"[Thị xã Phú Mỹ,, Bà Rịa - Vũng Tàu]",Bà Rịa - Vũng Tàu


In [52]:
career_link['Province or District'] = career_link['Location'].apply(lambda x: x[:-1] if x else None)

In [53]:
career_link['Province or District'] = career_link['Province or District'].apply(lambda x: ', '.join(x).rstrip(','))
career_link[['Location','Province or District','City']]

,Location,Province or District,City
0,"[Thành phố Huế,, Thừa Thiên - Huế]",Thành phố Huế,Thừa Thiên - Huế
1,"[Quận Cầu Giấy,, Hà Nội]",Quận Cầu Giấy,Hà Nội
2,"[Quận Tân Bình,, Hồ Chí Minh]",Quận Tân Bình,Hồ Chí Minh
3,"[Quận 4,, Hồ Chí Minh]",Quận 4,Hồ Chí Minh
4,"[Thị xã Điện Bàn,, Quảng Nam]",Thị xã Điện Bàn,Quảng Nam
...,...,...,...
22795,"[Quận 7,, Hồ Chí Minh]",Quận 7,Hồ Chí Minh
22796,"[Huyện Xuyên Mộc,, Bà Rịa - Vũng Tàu]",Huyện Xuyên Mộc,Bà Rịa - Vũng Tàu
22797,"[Huyện Xuyên Mộc,, Bà Rịa - Vũng Tàu]",Huyện Xuyên Mộc,Bà Rịa - Vũng Tàu
22798,"[Huyện Xuyên Mộc,, Bà Rịa - Vũng Tàu]",Huyện Xuyên Mộc,Bà Rịa - Vũng Tàu


In [54]:
for province in career_link['Province or District'].unique():
    print(province)

Thành phố Huế
Quận Cầu Giấy
Quận Tân Bình
Quận 4
Thị xã Điện Bàn
Thành phố Biên Hòa
Thành phố Từ Sơn
Thị xã Phú Mỹ
Thành Phố Thuận An
Thành Phố Thủ Đức
Huyện Cần Giuộc
Huyện Mê Linh
Quận Hoàng Mai

Huyện Sơn Tịnh
Huyện Tiên Du
Quận Liên Chiểu
Quận Bình Thạnh
Thành phố Việt Trì
Quận Hoàn Kiếm
Quận 1
Quận Ba Đình
Thành phố Thanh Hóa
Quận 3
Quận Hải Châu
Quận 10
Quận Tân Phú
Quận Bình Tân
Quận Ngũ Hành Sơn
Quận 4, Quận Tân Bình, Quận Phú Nhuận, Quận Bình Tân
Thành phố Tam Kỳ
Quận Nam Từ Liêm
Quận 8
Thị xã Sông Cầu
Huyện Phú Quốc
Quận 2
Huyện Yên Phong
Quận Thanh Xuân
Quận Đống Đa
Quận Phú Nhuận
Quận 2, Thành Phố Thủ Đức
Quận 5
Thành phố Hội An
Quận Long Biên
Quận Lê Chân
Quận Tây Hồ
Quận Tây Hồ, Huyện Đông Anh, Quận Hà Đông
Thành phố Đà Lạt
Thành phố Bà Rịa
Huyện An Dương
Thị xã An Khê
Huyện An Phú
Quận Bình Thủy
Thành phố Quy Nhơn
Thành phố Hải Dương
Thành phố Hạ Long
Quận Hồng Bàng, Quận Ngô Quyền, Quận Lê Chân
Quận Hồng Bàng
Quận Ngô Quyền
Thành Phố Phúc Yên
Huyện Thủy Nguyên
Quận Hai 

# Thay đổi thứ tự cột 

In [55]:
desired_columns = [
    'Job Title', 'Company', 'Company Link', 'Location', 'City', 'Province or District',
    'Gender', 'Female', 'Male',
    'Salary', 
    'Salary Type', 'Undefined Salary Type', 'Salary Start', 'Salary End', 'Currency Unit', 'Benefits',
    'Keywords', 'Job Summary', 'Description', 'Required Skills', 'Contact'
]

# Group 'Level' columns together
job_level_columns = [col for col in career_link.columns if col.startswith('Level')]
desired_columns.extend(job_level_columns)

# Group 'Education' columns together
job_education_columns = [col for col in career_link.columns if col.startswith('Education')]
desired_columns.extend(job_education_columns)

# Group 'Job Type' columns together
job_type_columns = [col for col in career_link.columns if col.startswith('Job Type')]
desired_columns.extend(job_type_columns)

# Group 'Experience' columns together
experience_columns = [col for col in career_link.columns if col.startswith('Experience')]
desired_columns.extend(experience_columns)

# Group 'Industry' columns together
industry_columns = [col for col in career_link.columns if col.startswith('Industry')]
desired_columns.extend(industry_columns)

career_link = career_link[desired_columns]

In [56]:
career_link

,Job Title,Company,Company Link,Location,City,Province or District,Gender,Female,Male,Salary,...,Industry_Quản lý điều hành,Industry_Báo chí / Biên tập viên / Xuất bản,Industry_Ngân hàng / Chứng khoán,Industry_Xuất nhập khẩu / Ngoại thương,Industry_Bảo trì / Sửa chữa,Industry_Khách sạn,Industry_Khác,Industry_Thư ký / Hành chánh,Industry_Vật tư / Thu mua,Industry_Viễn Thông
0,.Net Developer [Huế],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"[Thành phố Huế,, Thừa Thiên - Huế]",Thừa Thiên - Huế,Thành phố Huế,Nam / Nữ,1,1,Thương lượng,...,0,0,0,0,0,0,0,0,0,0
1,.Net Developer (Senior/Lead),CÔNG TY CỔ PHẦN SYNODUS,https://www.careerlink.vn//viec-lam-cua/cong-t...,"[Quận Cầu Giấy,, Hà Nội]",Hà Nội,Quận Cầu Giấy,Nam / Nữ,1,1,Trên 40 triệu,...,0,0,0,0,0,0,0,0,0,0
2,.Net Developer [Tân Bình],CÔNG TY TNHH SOFT WORLD VIỆT NAM,https://www.careerlink.vn//viec-lam-cua/cong-t...,"[Quận Tân Bình,, Hồ Chí Minh]",Hồ Chí Minh,Quận Tân Bình,Nam / Nữ,1,1,Thương lượng,...,0,0,0,0,0,0,0,0,0,0
3,.NET DEVELOPER,CÔNG TY CỔ PHẦN GIẢI PHÁP KINH DOANH NỀN TẢNG,https://www.careerlink.vn//viec-lam-cua/cong-t...,"[Quận 4,, Hồ Chí Minh]",Hồ Chí Minh,Quận 4,Nam / Nữ,1,1,Trên 600 USD,...,0,0,0,0,0,0,0,0,0,0
4,01 Chuyên Viên Quản Trị cơ sở dữ liệu/Database...,Rieker Vietnam Ltd.,https://www.careerlink.vn//viec-lam-cua/rieker...,"[Thị xã Điện Bàn,, Quảng Nam]",Quảng Nam,Thị xã Điện Bàn,Nam / Nữ,1,1,Thương lượng,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,Xuất nhập khẩu tiếng Nhật,G.A. CONSULTANTS VIETNAM,https://www.careerlink.vn//viec-lam-cua/g.a.-c...,"[Quận 7,, Hồ Chí Minh]",Hồ Chí Minh,Quận 7,Nam / Nữ,1,1,15 triệu - 25 triệu,...,0,0,0,0,0,0,0,0,0,0
22796,[XUYÊN MỘC] CHUYÊN VIÊN TƯ VẤN BẢO HIÊM- DỰ ÁN...,Sun Life Việt Nam (SLV),https://www.careerlink.vn//viec-lam-cua/sun-li...,"[Huyện Xuyên Mộc,, Bà Rịa - Vũng Tàu]",Bà Rịa - Vũng Tàu,Huyện Xuyên Mộc,Nam / Nữ,1,1,Trên 10 triệu,...,0,0,1,0,0,0,0,0,0,0
22797,[Xuyên Mộc] - Chuyên Viên Tư Vấn Bảo Hiểm Kênh...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"[Huyện Xuyên Mộc,, Bà Rịa - Vũng Tàu]",Bà Rịa - Vũng Tàu,Huyện Xuyên Mộc,Nam / Nữ,1,1,Cạnh tranh,...,0,0,1,0,0,0,0,0,0,0
22798,[Xuyên Mộc] Chuyên Viên Tư Vấn Bảo Hiểm Kênh L...,CÔNG TY TNHH MANULIFE (VIỆT NAM),https://www.careerlink.vn//viec-lam-cua/cong-t...,"[Huyện Xuyên Mộc,, Bà Rịa - Vũng Tàu]",Bà Rịa - Vũng Tàu,Huyện Xuyên Mộc,Nam / Nữ,1,1,10 triệu - 20 triệu,...,0,0,1,0,0,0,0,0,0,0


# Loại bỏ các cột dư thừa

In [57]:
career_link = career_link.drop(['Job Summary', 'Industry', 'Salary', 'Job Type', 'Experience', 'Gender', 'Level', 'Education', 'Location'], axis=1)

In [58]:
for col in career_link.columns:
    print(col)

Job Title
Company
Company Link
City
Province or District
Female
Male
Salary Type
Undefined Salary Type
Salary Start
Salary End
Currency Unit
Benefits
Keywords
Description
Required Skills
Contact
Level_Giám đốc
Level_Kỹ thuật viên / Kỹ sư
Level_Mới đi làm
Level_Nhân viên
Level_Quản lý / Trưởng phòng
Level_Quản lý cấp cao
Level_Sinh viên / Thực tập sinh
Level_Trưởng nhóm / Giám sát
Level_Điều hành cấp cao
Education_Cao đẳng
Education_Chứng chỉ
Education_Cử nhân
Education_Khác
Education_Kỹ sư
Education_Thạc sĩ
Education_Thạc sĩ Nghệ thuật
Education_Thạc sĩ QTKD
Education_Thạc sĩ Y học
Education_Tiến sĩ
Education_Trung cấp
Education_Trung học cơ sở
Education_Trung học phổ thông
Job Type_Nhân viên toàn thời gian
Job Type_Nhân viên toàn thời gian tạm thời
Job Type_Nhân viên bán thời gian
Job Type_Nhân viên hợp đồng
Job Type_Khác
Job Type_Nhân viên bán thời gian tạm thời
Experience Type (year)
Experience Start (year)
Experience End
Experience Undefined Type
Industry_CNTT - Phần mềm
Industry_Đ

In [59]:
career_link.shape

(22800, 106)